In [1]:
from pyspark.sql import functions as F
import pandas as pd

# Requires install scikit-learn==0.22.2.post1
from sklearn.impute import KNNImputer

In [2]:
df_races = spark.read.csv('/mnt/ne-gr5069/raw/races.csv',header=True)
display(df_races)

raceId,year,round,circuitId,name,date,time,url
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_Grand_Prix
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Grand_Prix
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Grand_Prix
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Grand_Prix
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Grand_Prix
6,2009,6,6,Monaco Grand Prix,2009-05-24,12:00:00,http://en.wikipedia.org/wiki/2009_Monaco_Grand_Prix
7,2009,7,5,Turkish Grand Prix,2009-06-07,12:00:00,http://en.wikipedia.org/wiki/2009_Turkish_Grand_Prix
8,2009,8,9,British Grand Prix,2009-06-21,12:00:00,http://en.wikipedia.org/wiki/2009_British_Grand_Prix
9,2009,9,20,German Grand Prix,2009-07-12,12:00:00,http://en.wikipedia.org/wiki/2009_German_Grand_Prix
10,2009,10,11,Hungarian Grand Prix,2009-07-26,12:00:00,http://en.wikipedia.org/wiki/2009_Hungarian_Grand_Prix


In [3]:
df_drivers = spark.read.csv('/mnt/ne-gr5069/raw/drivers.csv',header=True)
display(df_drivers)

driverId,driverRef,number,code,forename,surname,dob,nationality,url
1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
6,nakajima,\N,NAK,Kazuki,Nakajima,1985-01-11,Japanese,http://en.wikipedia.org/wiki/Kazuki_Nakajima
7,bourdais,\N,BOU,Sébastien,Bourdais,1979-02-28,French,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais
8,raikkonen,7,RAI,Kimi,Räikkönen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen
9,kubica,88,KUB,Robert,Kubica,1984-12-07,Polish,http://en.wikipedia.org/wiki/Robert_Kubica
10,glock,\N,GLO,Timo,Glock,1982-03-18,German,http://en.wikipedia.org/wiki/Timo_Glock


In [4]:
df_constructors = spark.read.csv('/mnt/ne-gr5069/raw/constructors.csv',header=True)
display(df_constructors)

constructorId,constructorRef,name,nationality,url
1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering
4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_in_Formula_One
5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso
6,ferrari,Ferrari,Italian,http://en.wikipedia.org/wiki/Scuderia_Ferrari
7,toyota,Toyota,Japanese,http://en.wikipedia.org/wiki/Toyota_Racing
8,super_aguri,Super Aguri,Japanese,http://en.wikipedia.org/wiki/Super_Aguri_F1
9,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing
10,force_india,Force India,Indian,http://en.wikipedia.org/wiki/Racing_Point_Force_India


In [5]:
df_qualifying = spark.read.csv('/mnt/ne-gr5069/raw/qualifying.csv',header=True)
display(df_qualifying)

qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
6,18,15,7,11,6,1:26.427,1:26.101,1:28.527
7,18,3,3,7,7,1:26.295,1:26.059,1:28.687
8,18,14,9,9,8,1:26.381,1:26.063,1:29.041
9,18,10,7,12,9,1:26.919,1:26.164,1:29.593
10,18,20,5,15,10,1:26.702,1:25.842,\N


In [6]:
df_results = spark.read.csv('/mnt/ne-gr5069/raw/results.csv',header=True)
display(df_results)

resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
1,18,1,1,22,1,1,1,1,10,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
2,18,2,2,3,5,2,2,2,8,58,+5.478,5696094,41,3,1:27.739,217.586,1
3,18,3,3,7,7,3,3,3,6,58,+8.163,5698779,41,5,1:28.090,216.719,1
4,18,4,4,5,11,4,4,4,5,58,+17.181,5707797,58,7,1:28.603,215.464,1
5,18,5,1,23,3,5,5,5,4,58,+18.014,5708630,43,1,1:27.418,218.385,1
6,18,6,3,8,13,6,6,6,3,57,\N,\N,50,14,1:29.639,212.974,11
7,18,7,5,14,17,7,7,7,2,55,\N,\N,22,12,1:29.534,213.224,5
8,18,8,6,1,15,8,8,8,1,53,\N,\N,20,4,1:27.903,217.180,5
9,18,9,2,4,2,\N,R,9,0,47,\N,\N,15,9,1:28.753,215.100,4
10,18,10,7,12,18,\N,R,10,0,43,\N,\N,23,13,1:29.558,213.166,3


In [7]:
df_results.count()

Out[8]: 24620

## Table generation

Unfortunately we cannot use the initial table inherited from
master table from inf features (df_inf_master). 

It removed data where where grid position is 0, referring to crashes,
DNQ, or car faults.

* Makes sense for inferential task - it is self-explanatory that 
DNQ/car fault means you can't win #2, so its inclusion is not theoretically
valuable.
* For prediction we want the model to pick this up on its own - removing
DNQ/car fault data from the test set based on statusId and just saying they
didn't get #2 feels like doing the model's work for it.

Also, it only includes races up to 2010

Hence, build data from scratch from df_results
* Filter to include only races from 1950 - 2017 (inclusive) - use df_races

Needed references to cross reference other tables and build features:
* Race ID
* Driver ID
* Constructor ID

Features already present:
* Starting Grid Position (need to deal with position 0 entries)


Features to build:
* **Outcome** - Number 2 - Build from positionOrder
* Constructor quality (Need to dummy code) - build from df_constructors 
and constructor champion list
* Driver Experience - build from df_races
* Driver Average Finishing Position - Build from df_results
* Constructor Average Finishing Position - Build from df_results
* Average Qualifying lap time - Build from df_qualifying
* Q3 Qualifying lap time - Build from df_qualifying

Save to S3

### Filter to include only races from 1950-2017 (inclusive)
### Drop irrelevant columns

In [10]:
df_results_filtered = df_results.select('raceId', 'driverId', 'constructorId', 'grid', 'positionOrder')\
  .join(df_races.select('raceId', 'year'), on = ['raceId'], how = 'left')

df_results_filtered = df_results_filtered\
  .filter(df_results_filtered['year'] >= 1950)\
  .filter(df_results_filtered['year'] <= 2017)
              
display(df_results_filtered)

raceId,driverId,constructorId,grid,positionOrder,year
18,1,1,1,1,2008
18,2,2,5,2,2008
18,3,3,7,3,2008
18,4,4,11,4,2008
18,5,1,3,5,2008
18,6,3,13,6,2008
18,7,5,17,7,2008
18,8,6,15,8,2008
18,9,2,2,9,2008
18,10,7,18,10,2008


In [11]:
df_results_filtered.count()

Out[10]: 23780

### Outcome - Number 2

#### Check Missing Values before if/else

In [14]:
display(df_results_filtered.groupby('positionOrder').count())
#display(df_results_filtered.groupby('positionOrder').count().agg(F.sum("count")))

# Looks good

positionOrder,count
7,983
15,971
11,984
29,180
3,986
30,156
34,46
8,983
22,716
28,221


#### Create outcome and drop inf outcomes

In [16]:
df_pred_w_outcome = df_results_filtered.withColumn('Second',
                                                   F.when(F.col('positionOrder') == 2, 1).otherwise(0))

display(df_pred_w_outcome)

raceId,driverId,constructorId,grid,positionOrder,year,Second
18,1,1,1,1,2008,0
18,2,2,5,2,2008,1
18,3,3,7,3,2008,0
18,4,4,11,4,2008,0
18,5,1,3,5,2008,0
18,6,3,13,6,2008,0
18,7,5,17,7,2008,0
18,8,6,15,8,2008,0
18,9,2,2,9,2008,0
18,10,7,18,10,2008,0


#### Missing Values and Join check

In [18]:
display(df_pred_w_outcome.groupby('Second').count())
df_pred_w_outcome.count()

# Looks good

Second,count
1,985
0,22795


### Grid Position
- Deal with 0 position entries by replacing them with the
positionOrder positions
  - This seems appropriate as the positionOrder values are the
inferred finishing position of these entries, tacked to the back of the
grid. This befits DNQs, crashouts, and technical faults.
  - None of the 0 positions have a podium finish, and only 11 finished in 
the top 15

In [20]:
df_pred_grid_fixed = df_pred_w_outcome.withColumn('grid_fixed',
                                                   F.when(F.col('grid') != '0', 
                                                          F.col('grid'))\
                                                  .otherwise(F.col('positionOrder')))

display(df_pred_grid_fixed)

raceId,driverId,constructorId,grid,positionOrder,year,Second,grid_fixed
18,1,1,1,1,2008,0,1
18,2,2,5,2,2008,1,5
18,3,3,7,3,2008,0,7
18,4,4,11,4,2008,0,11
18,5,1,3,5,2008,0,3
18,6,3,13,6,2008,0,13
18,7,5,17,7,2008,0,17
18,8,6,15,8,2008,0,15
18,9,2,2,9,2008,0,2
18,10,7,18,10,2008,0,18


#### Missing values and manipulation check

In [22]:
display(df_pred_grid_fixed.groupby('grid', 'grid_fixed').count())


# Here we are checking that the 0 grid values have been properly assigned to their
# position value and are not 0

# Looks good

grid,grid_fixed,count
34,34,1
15,15,965
18,18,906
28,28,30
0,34,45
17,17,940
5,5,983
1,1,987
24,24,429
0,19,20


In [23]:
df_pred_grid_fixed.count()

Out[16]: 23780

#### Remove grid and Position Order to prevent duplication and the outcome variable being in the features

In [25]:

df_pred_grid_fixed = df_pred_grid_fixed.drop('grid', 'positionOrder')
display(df_pred_grid_fixed)

raceId,driverId,constructorId,year,Second,grid_fixed
18,1,1,2008,0,1
18,2,2,2008,1,5
18,3,3,2008,0,7
18,4,4,2008,0,11
18,5,1,2008,0,3
18,6,3,2008,0,13
18,7,5,2008,0,17
18,8,6,2008,0,15
18,9,2,2008,0,2
18,10,7,2008,0,18


### Constructor Quality

In [27]:
# Because of the chassis and engine partnerships, etc. that create duplicates or counfounds
# for winning constructors, the wiki page links a Are the best way of pinpointing constructors
# Who have actually won in the past
constructors_won_urls = ['http://en.wikipedia.org/wiki/Scuderia_Ferrari',
                         'http://en.wikipedia.org/wiki/Williams_Grand_Prix_Engineering',
                         'http://en.wikipedia.org/wiki/McLaren',
                         'http://en.wikipedia.org/wiki/Team_Lotus',
                         'http://en.wikipedia.org/wiki/Mercedes-Benz_in_Formula_One',
                         'http://en.wikipedia.org/wiki/Red_Bull_Racing',
                         'http://en.wikipedia.org/wiki/Cooper_Car_Company',
                         'http://en.wikipedia.org/wiki/Brabham',
                         'http://en.wikipedia.org/wiki/Renault_in_Formula_One',
                         'http://en.wikipedia.org/wiki/Vanwall',
                         'http://en.wikipedia.org/wiki/BRM',
                         'http://en.wikipedia.org/wiki/Matra',
                         'http://en.wikipedia.org/wiki/Tyrrell_Racing',
                         'http://en.wikipedia.org/wiki/Benetton_Formula',
                         'http://en.wikipedia.org/wiki/Brawn_GP']

constructors_won = df_constructors.where(F.col("url").isin(constructors_won_urls)).toPandas()
constructors_won = [int(id) for id in constructors_won['constructorId'].tolist()]

In [28]:
df_pred_grid_fixed_const = df_pred_grid_fixed\
  .withColumn('constructor_quality',
              F.when(F.col('constructorId').isin(constructors_won), 'winner')\
              .otherwise('not_winner'))

display(df_pred_grid_fixed_const)

raceId,driverId,constructorId,year,Second,grid_fixed,constructor_quality
18,1,1,2008,0,1,winner
18,2,2,2008,1,5,not_winner
18,3,3,2008,0,7,winner
18,4,4,2008,0,11,winner
18,5,1,2008,0,3,winner
18,6,3,2008,0,13,winner
18,7,5,2008,0,17,not_winner
18,8,6,2008,0,15,winner
18,9,2,2008,0,2,not_winner
18,10,7,2008,0,18,not_winner


#### Missing Values and Joins

In [30]:
# Check for missing constructor IDs
# display(df_constructors.groupby('constructorId').count())
#display(df_pred_grid_fixed_const.groupby('constructorId').count())
#display(df_pred_grid_fixed_const.groupby('constructor_quality').count())
display(df_pred_grid_fixed_const.groupby('constructorId').count().agg(F.sum("count")))

sum(count)
23780


### Driver Experience

In [32]:
driver_debut_year = df_pred_grid_fixed_const\
  .groupby('driverId')\
  .agg(F.min(F.col('year')).alias('debut_year'))
display(driver_debut_year)

driverId,debut_year
296,1972
467,1961
675,1951
691,1953
829,2014
125,1992
451,1963
800,1950
666,1950
124,1993


In [33]:
df_pred_grid_fixed_const_driver_debut = df_pred_grid_fixed_const.join(driver_debut_year, on = ['driverId'], how = 'left')\
  .withColumn("years_since_debut", F.col('year') - F.col("debut_year"))\
  .drop('debut_year')
display(df_pred_grid_fixed_const_driver_debut)

driverId,raceId,constructorId,year,Second,grid_fixed,constructor_quality,years_since_debut
296,579,26,1975,0,22,not_winner,3.0
296,581,75,1975,0,22,not_winner,3.0
296,584,3,1975,0,24,winner,3.0
296,590,66,1974,0,24,winner,2.0
296,591,66,1974,0,23,winner,2.0
296,592,66,1974,0,25,winner,2.0
296,593,66,1974,0,22,winner,2.0
296,594,66,1974,0,25,winner,2.0
296,595,66,1974,0,22,winner,2.0
296,597,66,1974,0,25,winner,2.0


#### Missing Values and Joins

In [35]:
# Check for missing driver IDs
#display(df_pred_grid_fixed_const_driver_debut.groupby('driverId').count())
display(df_pred_grid_fixed_const_driver_debut.groupby('driverId').count().agg(F.sum("count")))

# Looks good

sum(count)
23780


### Driver Average Finishing Position

In [37]:
driver_avg_finish = df_results\
  .groupby('driverId')\
  .agg(F.mean(F.col('positionOrder')).alias('driver_avg_finish'))
display(driver_avg_finish)

driverId,driver_avg_finish
296,18.066666666666666
467,32.0
675,12.25
691,8.5
829,16.63157894736842
125,16.0
451,22.0
800,7.0
666,17.0
7,14.148148148148149


In [38]:
df_pred_grid_fixed_const_driver_debut_finish = df_pred_grid_fixed_const_driver_debut\
  .join(driver_avg_finish, on = ['driverId'], how = 'left')
display(df_pred_grid_fixed_const_driver_debut_finish)

driverId,raceId,constructorId,year,Second,grid_fixed,constructor_quality,years_since_debut,driver_avg_finish
296,579,26,1975,0,22,not_winner,3.0,18.066666666666666
296,581,75,1975,0,22,not_winner,3.0,18.066666666666666
296,584,3,1975,0,24,winner,3.0,18.066666666666666
296,590,66,1974,0,24,winner,2.0,18.066666666666666
296,591,66,1974,0,23,winner,2.0,18.066666666666666
296,592,66,1974,0,25,winner,2.0,18.066666666666666
296,593,66,1974,0,22,winner,2.0,18.066666666666666
296,594,66,1974,0,25,winner,2.0,18.066666666666666
296,595,66,1974,0,22,winner,2.0,18.066666666666666
296,597,66,1974,0,25,winner,2.0,18.066666666666666


#### Missing Data and Joins

In [40]:
# Check for missing driver IDs
# display(df_pred_grid_fixed_const_driver_debut_finish.groupby('driverId').count())
# display(df_pred_grid_fixed_const_driver_debut_finish.groupby('driverId').count().agg(F.sum("count")))
display(df_pred_grid_fixed_const_driver_debut_finish.describe('driver_avg_finish'))

# Looks Good

summary,driver_avg_finish
count,23780
mean,13.077725116302847
stddev,4.266399378772213
min,2.0
max,38.833333333333336


### Constructor Average Finishing Position

In [42]:
const_avg_finish = df_results\
  .groupby('constructorId')\
  .agg(F.mean(F.col('positionOrder')).alias('const_avg_finish'))
display(const_avg_finish)

constructorId,const_avg_finish
125,16.40740740740741
7,11.45
51,13.30877192982456
124,13.0
169,13.0
205,17.973684210526315
15,12.901140684410647
54,18.25925925925926
155,19.0
132,12.0


In [43]:
df_pred_wo_quals = df_pred_grid_fixed_const_driver_debut_finish\
  .join(const_avg_finish, on = ['constructorId'], how = 'left')
display(df_pred_wo_quals)

constructorId,driverId,raceId,year,Second,grid_fixed,constructor_quality,years_since_debut,driver_avg_finish,const_avg_finish
26,296,579,1975,0,22,not_winner,3.0,18.066666666666666,16.47878787878788
75,296,581,1975,0,22,not_winner,3.0,18.066666666666666,13.947368421052632
3,296,584,1975,0,24,winner,3.0,18.066666666666666,10.54477101845523
66,296,590,1974,0,24,winner,2.0,18.066666666666666,12.594306049822064
66,296,591,1974,0,23,winner,2.0,18.066666666666666,12.594306049822064
66,296,592,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064
66,296,593,1974,0,22,winner,2.0,18.066666666666666,12.594306049822064
66,296,594,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064
66,296,595,1974,0,22,winner,2.0,18.066666666666666,12.594306049822064
66,296,597,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064


#### Missing Data

In [45]:
# Check for missing driver IDs
# display(df_pred_wo_quals.groupby('constructorId').count())
# display(df_pred_wo_quals.groupby('constructorId').count().agg(F.sum("count")))
display(df_pred_wo_quals.describe('const_avg_finish'))

# Looks good

summary,const_avg_finish
count,23780
mean,13.079770534293187
stddev,3.901387000732917
min,4.882352941176471
max,35.0


### Q3 Qualifying Lap Time

Replace missing values with none

In [48]:
df_qualifying_missing_fix = df_qualifying.withColumn('q1_fix',
                                                     F.when(F.col('q1') == '\\N', None)\
                                                     .otherwise(F.col('q1')))
display(df_qualifying_missing_fix)

qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,q1_fix
1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,1:26.572
2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,1:26.103
3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,1:25.664
4,18,13,6,2,4,1:25.994,1:25.691,1:27.178,1:25.994
5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,1:25.960
6,18,15,7,11,6,1:26.427,1:26.101,1:28.527,1:26.427
7,18,3,3,7,7,1:26.295,1:26.059,1:28.687,1:26.295
8,18,14,9,9,8,1:26.381,1:26.063,1:29.041,1:26.381
9,18,10,7,12,9,1:26.919,1:26.164,1:29.593,1:26.919
10,18,20,5,15,10,1:26.702,1:25.842,\N,1:26.702


Turn times to milliseconds

In [50]:
split_q1 = F.split(df_qualifying_missing_fix['q1_fix'], '\D')

qual_times = df_qualifying_missing_fix\
  .withColumn('q1_ms', (split_q1.getItem(0) * 60 * 1000) + (split_q1.getItem(1) * 1000) + (split_q1.getItem(2)))\
  .select('raceId', 'driverId', 'q1_ms')

display(qual_times)

raceId,driverId,q1_ms
18,1,86572.0
18,9,86103.0
18,5,85664.0
18,13,85994.0
18,2,85960.0
18,15,86427.0
18,3,86295.0
18,14,86381.0
18,10,86919.0
18,20,86702.0


Check to see if values make sense

In [52]:
display(qual_times.describe())

summary,raceId,driverId,q1_ms
count,8354,8354,8220
mean,505.9713909504429,246.82643045247787,89312.66435523114
stddev,401.19495313139873,351.59209887237796,15891.53573543267
min,1,1,63807.0
max,999,99,1002640.0


Only 8000+ entries suggests something is wrong. I suspect like pitstops, we only have 
qualifying data from more recent years

In [54]:
display(qual_times.join(df_races.select('raceId', 'year'), on = ['raceId'], how = 'left').describe())

summary,raceId,driverId,q1_ms,year
count,8354,8354,8220,8354
mean,505.9713909504429,246.82643045247787,89312.66435523114,2008.482762748384
stddev,401.19495313139873,351.59209887237796,15891.53573543267,7.358774656709036
min,1,1,63807.0,1994
max,999,99,1002640.0,2019


Ok I should have checked this earlier, and thankfully I did check, we only have 
qualifying lap data from 1994 onwards. This is a problem as the majority of our
training set will be missing this feature.

I think imputation is going to be a problem, because chances are races from earlier in
F1 history have slower laptimes. Missingness is far from random here. 

There is also a small amount of missing data in q1_ms, likely to be people who didn't 
even manage to complete q1 qualifying for some reason. This is a small number, and
imputation might work here given the relationship with the fixed grid position.

As for the earlier races, it's safest to do without the feature, but we can try to
impute the values with the other features and see which works better. In effect,
we use that as an additional iteration parameter.

In [56]:
df_pred_full = df_pred_wo_quals.join(qual_times, on = ['raceId', 'driverId'], how = 'left')

display(df_pred_full)

raceId,driverId,constructorId,year,Second,grid_fixed,constructor_quality,years_since_debut,driver_avg_finish,const_avg_finish,q1_ms
579,296,26,1975,0,22,not_winner,3.0,18.066666666666666,16.47878787878788,null
581,296,75,1975,0,22,not_winner,3.0,18.066666666666666,13.947368421052632,null
584,296,3,1975,0,24,winner,3.0,18.066666666666666,10.54477101845523,null
590,296,66,1974,0,24,winner,2.0,18.066666666666666,12.594306049822064,null
591,296,66,1974,0,23,winner,2.0,18.066666666666666,12.594306049822064,null
592,296,66,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064,null
593,296,66,1974,0,22,winner,2.0,18.066666666666666,12.594306049822064,null
594,296,66,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064,null
595,296,66,1974,0,22,winner,2.0,18.066666666666666,12.594306049822064,null
597,296,66,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064,null


In [57]:
pd_df_pred_full = df_pred_full.toPandas()

In [58]:
pd_df_pred_full.head()

,raceId,driverId,constructorId,year,Second,grid_fixed,constructor_quality,years_since_debut,driver_avg_finish,const_avg_finish,q1_ms
0,579,296,26,1975,0,22,not_winner,3.0,18.066667,16.478788,NaN
1,581,296,75,1975,0,22,not_winner,3.0,18.066667,13.947368,NaN
2,584,296,3,1975,0,24,winner,3.0,18.066667,10.544771,NaN
3,590,296,66,1974,0,24,winner,2.0,18.066667,12.594306,NaN
4,591,296,66,1974,0,23,winner,2.0,18.066667,12.594306,NaN


In [59]:
df_4_impute = pd.get_dummies(pd_df_pred_full.drop(['raceId', 'driverId', 'constructorId', 'year', 'Second'],
                                                  axis = 1),
                             columns = ['constructor_quality'])

knn_imputer = KNNImputer(n_neighbors=1)

array_missing_imputed = knn_imputer.fit_transform(df_4_impute)
df_missing_imputed = pd.DataFrame(array_missing_imputed, columns = df_4_impute.columns)

In [60]:
pd_df_pred_full_w_impute = pd_df_pred_full.merge(df_missing_imputed[['q1_ms']],
                                                 how='left',
                                                 left_index=True,
                                                 right_index=True,
                                                 sort=False,
                                                 suffixes=('_OG', '_impute'))

In [61]:
pd_df_pred_full_w_impute.describe(include = 'all')

,raceId,driverId,constructorId,year,Second,grid_fixed,constructor_quality,years_since_debut,driver_avg_finish,const_avg_finish,q1_ms_OG,q1_ms_impute
count,23780,23780,23780,23780,23780.000000,23780,23780,23780.000000,23780.000000,23780.000000,7.397000e+03,2.378000e+04
unique,976,842,207,68,NaN,39,2,NaN,NaN,NaN,NaN,NaN
top,800,22,6,1989,NaN,1,not_winner,NaN,NaN,NaN,NaN,NaN
freq,55,326,2141,620,NaN,987,12050,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,0.041421,NaN,NaN,4.304794,13.077725,13.079771,8.974245e+04,9.084304e+04
std,NaN,NaN,NaN,NaN,0.199267,NaN,NaN,3.862316,4.266399,3.901387,1.625338e+04,2.313690e+04
min,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,2.000000,4.882353,6.506400e+04,6.506400e+04
25%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000,10.167598,10.544771,8.022900e+04,8.087775e+04
50%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,3.000000,12.359375,12.970588,8.760900e+04,8.788100e+04
75%,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,7.000000,15.231579,14.666667,9.721000e+04,9.631725e+04


## Train/Test Split

In [63]:
spark_df_pred_full_w_impute = spark_df_missing_imputed = spark.createDataFrame(pd_df_pred_full_w_impute)
display(spark_df_pred_full_w_impute)

raceId,driverId,constructorId,year,Second,grid_fixed,constructor_quality,years_since_debut,driver_avg_finish,const_avg_finish,q1_ms_OG,q1_ms_impute
579,296,26,1975,0,22,not_winner,3.0,18.066666666666666,16.47878787878788,null,95651.0
581,296,75,1975,0,22,not_winner,3.0,18.066666666666666,13.947368421052632,null,82513.0
584,296,3,1975,0,24,winner,3.0,18.066666666666666,10.54477101845523,null,109809.0
590,296,66,1974,0,24,winner,2.0,18.066666666666666,12.594306049822064,null,148979.0
591,296,66,1974,0,23,winner,2.0,18.066666666666666,12.594306049822064,null,148979.0
592,296,66,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064,null,85313.0
593,296,66,1974,0,22,winner,2.0,18.066666666666666,12.594306049822064,null,148979.0
594,296,66,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064,null,85313.0
595,296,66,1974,0,22,winner,2.0,18.066666666666666,12.594306049822064,null,148979.0
597,296,66,1974,0,25,winner,2.0,18.066666666666666,12.594306049822064,null,85313.0


In [64]:
df_pred_train = spark_df_pred_full_w_impute.filter(spark_df_pred_full_w_impute['year'] <= 2010)
display(df_pred_train.describe('year'))

summary,year
count,20779
mean,1983.8816112421193
stddev,15.975795599314399
min,1950
max,2010


In [65]:
df_pred_test = spark_df_pred_full_w_impute.filter(spark_df_pred_full_w_impute['year'] > 2010)
display(df_pred_test.describe('year'))

summary,year
count,3001
mean,2013.9186937687437
stddev,2.0205742813676046
min,2011
max,2017


## Save to S3

In [67]:
df_pred_train.coalesce(1).write.csv('/mnt/xql2001-gr5069/interim/final_project/df_pred_train.csv', header=True)
df_pred_test.coalesce(1).write.csv('/mnt/xql2001-gr5069/interim/final_project/df_pred_test.csv', header=True)